##Предобработка и соединение таблиц


In [3]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

In [4]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv').sample(60000)
user_df = pd.read_csv('Users.csv')

<ipython-input-4-1407295537a8>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv('Books.csv')


In [5]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [6]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60000 entries, 10965 to 686056
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   User-ID      60000 non-null  int64 
 1   ISBN         60000 non-null  object
 2   Book-Rating  60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


In [7]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [8]:
ratings_df.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


In [9]:
ratings_df = ratings_df[ratings_df['Book-Rating'] != 0]

In [10]:
book_df.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2
Image-URL-S,0
Image-URL-M,0
Image-URL-L,3


In [11]:
book_df.dropna(how='any', inplace=True)

In [12]:
user_df.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


In [13]:
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

In [14]:
ratings_df.head(10)

,User-ID,ISBN,Book-Rating
300757,71612,9725642317,10
376077,90498,8845231836,10
723576,174892,0373263082,6
592956,143163,087011249X,10
978262,235342,8845251209,8
602668,145527,2742728783,6
1106753,265487,0440236355,5
241630,55550,0345300882,8
865783,209385,0451173376,8
740572,179019,0385492081,9


In [15]:
user_rating_df.head(10)

,User-ID,ISBN,Book-Rating,Location,Age
0,71612,9725642317,10,"covilhã, covilhã, portugal",25.0
1,90498,8845231836,10,"torino, ,",NaN
2,174892,0373263082,6,"milford, connecticut,",NaN
3,143163,087011249X,10,"weaverville, california, usa",26.0
4,235342,8845251209,8,"nitra, nitriansky kraj, slovakia",33.0
5,145527,2742728783,6,"aylmer, quebec, canada",52.0
6,265487,0440236355,5,"kaneohe, hawaii, usa",75.0
7,55550,0345300882,8,"northern, virginia, usa",42.0
8,209385,0451173376,8,"wyndmoor, pennsylvania, usa",57.0
9,179019,0385492081,9,"little falls, new york,",NaN


In [16]:
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN', right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace=True) # сброс индексов
book_user_rating.head(10)

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0060973129,Decision in Normandy,Carlo D'Este,152827,7
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,197659,9
2,0440234743,The Testament,John Grisham,161870,5
3,0440234743,The Testament,John Grisham,244761,6
4,0440234743,The Testament,John Grisham,67487,6
5,0440234743,The Testament,John Grisham,125519,8
6,0440234743,The Testament,John Grisham,86969,8
7,0440234743,The Testament,John Grisham,233731,7
8,0440234743,The Testament,John Grisham,143134,10
9,0440234743,The Testament,John Grisham,250101,9


In [17]:
book_user_rating.size

100645

Создаем новые уникальные идентификаторы

In [18]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)
book_user_rating.head(10)

<ipython-input-18-0109687a50a9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)


,ISBN,Book-Title,Book-Author,User-ID,Book-Rating,unique_id_book
0,0060973129,Decision in Normandy,Carlo D'Este,152827,7,0
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,197659,9,1
2,0440234743,The Testament,John Grisham,161870,5,2
3,0440234743,The Testament,John Grisham,244761,6,2
4,0440234743,The Testament,John Grisham,67487,6,2
5,0440234743,The Testament,John Grisham,125519,8,2
6,0440234743,The Testament,John Grisham,86969,8,2
7,0440234743,The Testament,John Grisham,233731,7,2
8,0440234743,The Testament,John Grisham,143134,10,2
9,0440234743,The Testament,John Grisham,250101,9,2


##Рекомендательная система

Эта рекомендательная система использует метод факторизации матриц для создания рекомендаций книг. Основная идея заключается в выявлении скрытых факторов, которые определяют предпочтения пользователей на основе их предыдущих оценок.

Создаем сводную таблицу, где строки это пользователи, столбцы - книги, а значения - оценки

In [19]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID',
                                                        columns='unique_id_book',
                                                        values='Book-Rating').fillna(0)
users_books_pivot_matrix_df.head(10)

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,15643,15644,15645,15646,15647,15648,15649,15650,15651,15652
User-ID,,,,,,,,,,,,,,,,,,,,,
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Преобразовываем в массив numpy для

In [20]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])



```
# Выбран кодовый формат
```

Разложение исходной матрицы, это позволит нам выявить скрытые факторы, влияющие на оценки пользователями книг. Для этого мы используем метод сингулярного заложения матрицы.

In [21]:
NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [22]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

Двойное матричное умножение для получения предположительных оценок от каждого пользователя к каждой книге

In [23]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

array([[ 5.70195539e-32,  2.05162283e-18, -1.59199580e-20, ...,
        -1.26668652e-33, -1.26668652e-33,  2.70204857e-21],
       [-3.20084063e-34, -7.97853065e-18,  1.10187338e-19, ...,
         2.05276204e-34,  2.05276204e-34, -4.28070179e-19],
       [-4.31231150e-33, -6.12552767e-18,  5.93492353e-21, ...,
         6.85471605e-35,  6.85471605e-35, -1.36930650e-19],
       ...,
       [ 6.87277713e-33,  4.89692987e-18,  7.95974776e-20, ...,
        -6.66812097e-35, -6.66812097e-35,  1.51978940e-19],
       [ 8.77070952e-22, -7.95318050e-08, -1.10829620e-09, ...,
         3.80358588e-23,  3.80358588e-23, -9.79521703e-08],
       [ 8.72529421e-33,  1.00340267e-17,  6.91785221e-21, ...,
        -1.27285713e-34, -1.27285713e-34,  2.21908993e-19]])

Функция top_cosine_similarity вычисляет косинусное сходство между выбранной книгой и всеми остальными книгами. Косинусное сходство позволяет определить, насколько близки векторы книг друг к другу (векторы представляют книги в многомерном пространстве факторов, полученных с помощью SVD). Чем выше косинусное сходство, тем более похожими считаются книги.

In [24]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)

    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])

In [29]:
k = 50
book_id =545
top_n = 3
sliced = Vt.T[:, :k] # representative data

similar_books(book_user_rating, 545, top_cosine_similarity(sliced, book_id, top_n))

Recommendations for Rachel's Holiday: 

Little Girl Lost
Another Man's Wife
When Night Draws Near (Harlequin Intrigue, No. 540)
